In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
# import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
test.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [7]:
train.Age.fillna(train.Age.mean(), inplace=True)
test.Age.fillna(test.Age.mean(), inplace=True)

In [8]:
train.Cabin.fillna(train.Cabin.mode()[0], inplace=True)
test.Cabin.fillna(test.Cabin.mode()[0], inplace=True)

In [9]:
train.Embarked.fillna(train.Embarked.mode()[0], inplace=True)

In [10]:
test.Fare.fillna(train.Fare.mean(), inplace=True)

In [11]:
test.isna().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [12]:
cat_col = []
for col in train.columns:
    if train[col].dtype == "object":
        cat_col.append(col)

#LabelEncoder
lb = LabelEncoder()
for column in cat_col:
    train[column] = lb.fit_transform(train[column])
    test[column] = lb.fit_transform(test[column])

In [13]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,108,1,22.0,1,0,523,7.2500,47,2
1,2,1,1,190,0,38.0,1,0,596,71.2833,81,0
2,3,1,3,353,0,26.0,0,0,669,7.9250,47,2
3,4,1,1,272,0,35.0,1,0,49,53.1000,55,2
4,5,0,3,15,1,35.0,0,0,472,8.0500,47,2


In [14]:
X = train.drop(['Survived', 'PassengerId'], axis=1)
y = train['Survived']

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=0)

In [16]:
import lightgbm as lgb
lgb_train = lgb.Dataset(X_train, y_train,free_raw_data=False)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train,free_raw_data=False)

In [17]:
params = {
        'task': 'train',
        'objective': 'binary',
        'metric': 'binary_error',
        'verbose': 1
}

In [18]:
params1 = {   'learning_rate': 0.1, 'n_estimators': 5000, 'boosting_type': 'gbdt', 'reg_sqrt': True,
                  'objective': 'mse', 'sub_feature': 1.0, 'num_leaves': 8, 'min_data_in_leaf': 10,
                  'subsample_freq': 1, 'subsample': .5, 'reg_lambda': 0.1, 'reg_alpha': 0.1, 'random_state': 1,
                  'max_bin': 200, 'n_jobs': -1, 'verbose': -1
              }

In [19]:
gbm = lgb.train(params1,
            lgb_train,
            num_boost_round=1000,
            early_stopping_rounds=30,
            valid_sets=[lgb_train,lgb_eval],
            valid_names=['train', 'eval'])

[1]	train's l2: 0.279011	eval's l2: 0.268757
Training until validation scores don't improve for 30 rounds.
[2]	train's l2: 0.263016	eval's l2: 0.253901
[3]	train's l2: 0.248388	eval's l2: 0.239876
[4]	train's l2: 0.236534	eval's l2: 0.228294
[5]	train's l2: 0.225378	eval's l2: 0.218267
[6]	train's l2: 0.214011	eval's l2: 0.20813
[7]	train's l2: 0.204781	eval's l2: 0.200287
[8]	train's l2: 0.196551	eval's l2: 0.192942
[9]	train's l2: 0.189498	eval's l2: 0.186809
[10]	train's l2: 0.184334	eval's l2: 0.18226
[11]	train's l2: 0.178401	eval's l2: 0.176622
[12]	train's l2: 0.173238	eval's l2: 0.171094
[13]	train's l2: 0.167097	eval's l2: 0.165257
[14]	train's l2: 0.162278	eval's l2: 0.160689
[15]	train's l2: 0.158988	eval's l2: 0.158468
[16]	train's l2: 0.155494	eval's l2: 0.156236
[17]	train's l2: 0.153386	eval's l2: 0.156839
[18]	train's l2: 0.150618	eval's l2: 0.154434
[19]	train's l2: 0.14935	eval's l2: 0.154329
[20]	train's l2: 0.146787	eval's l2: 0.153242
[21]	train's l2: 0.144503	eval

In [20]:
prediction = gbm.predict(X_test)

In [21]:
encodedPrediction = [1 if x > 0.50 else 0 for x in prediction]    

In [22]:
print(accuracy_score(y_test, encodedPrediction))
# print(confusion_matrix(y_test, encodedPrediction))

0.835820895522388


In [23]:
features = list(X.columns)

# Export via Extended Nyoka

In [25]:
from nyoka import model_to_pmml

In [26]:
toExportDict={
    'model1':{
        'hyperparameters':params1,
        'preProcessingScript':None,
        'pipelineObj':None,
        'modelObj':gbm,
        'featuresUsed':features,
        'targetName':'Survived',
        'postProcessingScript':None,
        'taskType': 'trainAndscore'
    }
}

pmml = model_to_pmml(toExportDict, pmml_f_name="lgbmTrain.pmml")

# Export to PMML

In [ ]:
import lgbmTrainingAPI_to_pmml

In [ ]:
filename = 'lgbmExportedPMM.pmml'

In [ ]:
lgbmTrainingAPI_to_pmml.Export(model = gbm , pmml_f_name = filename)

In [ ]:
# jsondata = gbm.dump_model()
# import pprint
# pp = pprint.PrettyPrinter()
# pp.pprint(jsondata)

# Export to Text

In [ ]:
gbm.save_model('textModel.txt')

# FILE RECONSTRUCT (textModel.txt)

In [ ]:
newgbm = lgb.basic.Booster(params = {'model_str' : open('textModel.txt', "r").read()})

In [ ]:
frcprediction = newgbm.predict(X_test)

In [ ]:
frcencodedPrediction = [1 if x > 0.50 else 0 for x in frcprediction] 

In [ ]:
print(accuracy_score(y_test, frcencodedPrediction))
# print(confusion_matrix(y_test, frcencodedPrediction))

# PMML RECONSTRUCT

In [ ]:
import pmml_to_lgbmTrainAPI_Dev

In [ ]:
reconstructed_model = pmml_to_lgbmTrainAPI_Dev.reconstruct(filename)

In [ ]:
rcprediction = reconstructed_model.predict(X_test)

In [ ]:
rcencodedPrediction = [1 if x > 0.50 else 0 for x in rcprediction]

In [ ]:
print(accuracy_score(y_test, rcencodedPrediction))
print(confusion_matrix(y_test, rcencodedPrediction))

In [ ]:
encodedPrediction == rcencodedPrediction

# Continue training

In [ ]:
contGbm = lgb.train(params,
            lgb_train,
            num_boost_round=1000,
            early_stopping_rounds=30,
            init_model=reconstructed_model,
            valid_sets=[lgb_train,lgb_eval],
            valid_names=['train', 'eval'])

In [ ]:
continuedModelprediction = reconstructed_model.predict(X_test)

In [ ]:
rcencodedPrediction = [1 if x > 0.50 else 0 for x in rcprediction]

In [ ]:
print(accuracy_score(y_test, rcencodedPrediction))
print(confusion_matrix(y_test, rcencodedPrediction))

In [ ]:
type(contGbm)

In [ ]:
contGbm.__class__.__name__

In [ ]:
str(contGbm)